## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-31-10-50-54 +0000,nyt,"Thailand and Cambodia Reach Trump Trade Deals,...",https://www.nytimes.com/2025/07/31/business/th...
1,2025-07-31-10-38-00 +0000,wsj,Stock Market Today: S&P 500 Futures Rise After...,https://www.wsj.com/livecoverage/stock-market-...
2,2025-07-31-10-37-06 +0000,bbc,Canada follows France and UK with plan to reco...,https://www.bbc.com/news/articles/ceqyx35d9x2o...
3,2025-07-31-10-36-05 +0000,bbc,"Dozens killed seeking food in Gaza, hospital s...",https://www.bbc.com/news/articles/c74d82pdxjzo...
4,2025-07-31-10-33-13 +0000,cbc,Trump says U.S. will set 15% tariff on South K...,https://www.cbc.ca/news/world/trump-tariffs-so...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2302/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,70
4,trade,20
56,tariffs,19
36,new,18
20,palestinian,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
167,2025-07-30-22-42-00 +0000,nypost,Trump announces South Korea trade deal with $3...,https://nypost.com/2025/07/30/us-news/trump-an...,155
4,2025-07-31-10-33-13 +0000,cbc,Trump says U.S. will set 15% tariff on South K...,https://www.cbc.ca/news/world/trump-tariffs-so...,141
156,2025-07-30-23-08-32 +0000,cbc,Trump slams Canada's plan to recognize Palesti...,https://www.cbc.ca/news/politics/us-canada-tra...,140
6,2025-07-31-10-31-23 +0000,wapo,Trump puts trade deal in doubt after Canada ba...,https://www.washingtonpost.com/politics/2025/0...,140
299,2025-07-30-16-33-22 +0000,bbc,Trump to hit India with 25% tariffs - plus 'pe...,https://www.bbc.com/news/articles/cz71w1dgl84o...,138


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
167,155,2025-07-30-22-42-00 +0000,nypost,Trump announces South Korea trade deal with $3...,https://nypost.com/2025/07/30/us-news/trump-an...
241,93,2025-07-30-20-19-00 +0000,wsj,Former Vice President Kamala Harris said Wedne...,https://www.wsj.com/politics/elections/kamala-...
340,65,2025-07-30-13-51-11 +0000,cbc,Bank of Canada holds interest rate at 2.75% as...,https://www.cbc.ca/news/business/boc-july-deci...
126,61,2025-07-31-00-53-35 +0000,nypost,NYC shooting victim Julia Hyman remembered as ...,https://nypost.com/2025/07/30/us-news/nyc-shoo...
94,50,2025-07-31-03-54-00 +0000,wsj,Senators rejected an effort to block U.S. arms...,https://www.wsj.com/politics/policy/republican...
223,48,2025-07-30-20-52-58 +0000,wapo,Fed keeps rates steady but warns about slowing...,https://www.washingtonpost.com/business/2025/0...
15,47,2025-07-31-10-00-00 +0000,nypost,Jilted Barneys heir alleges family evaded $20M...,https://nypost.com/2025/07/31/business/jilted-...
283,47,2025-07-30-17-43-00 +0000,wsj,Tsunami Waves Reach U.S. After Powerful Russia...,https://www.wsj.com/world/tsunami-watch-warnin...
351,44,2025-07-30-12-53-59 +0000,bbc,Lammy hopes plan to recognise Palestinian stat...,https://www.bbc.com/news/articles/c23p8gl05r2o...
144,44,2025-07-30-23-37-48 +0000,wapo,NTSB hearing: Army helicopter’s location syste...,https://www.washingtonpost.com/business/2025/0...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
